In [1]:
using Plots
using Distributions
using SpecialFunctions
using Random
using DataFrames
using Combinatorics
using RobustMeans

In [3]:
# Moyenne théorique = 0.1
data1 = rand(Exponential(0.1), 1000000)

function median_of_means_opt_C(X)
    
    alpha=0.025
    medians = []
    for c in 1:40
        n = length(X)
        k = trunc(Int, c * log(1 / alpha))
        m = div(n, k)
        indic = repeat(1:k, inner = m)
        indic = indic[randperm(length(indic))]
        means = [mean(X[findall(indic .== block)]) for block in 1:k]
        push!(medians, median(means))
    end
    return medians
end


#Fonction qui calcul la borne inf et sup ainsi que l'écart entre la moyenne et l'estimateur
function IC_MOM2(X, μ, σ)
    alpha = 0.025
    μ_MOM = median_of_means_opt_C(X)
    inf_vals = Float64[]
    sup_vals = Float64[]
    c_vals = Float64[]
    ecart = abs.(μ_MOM .- μ)

    for c in sqrt(2)/4:0.01:10
        sup = σ * sqrt(4 * c * log(1 / alpha) / length(X))
        inf = -σ * sqrt(4 * c * log(1 / alpha) / length(X))
        push!(inf_vals, inf)
        push!(sup_vals, sup)
        push!(c_vals, c)
    end

    return inf_vals, sup_vals, c_vals, ecart
end

IC_MOM2 (generic function with 1 method)

In [4]:
#Application de la fonction IC_MOM2 sur une loi Exponentiel

# Exécutez d'abord la fonction IC_MOM2 pour obtenir les valeurs de retour
inf_vals, sup_vals, c_vals, ecart = IC_MOM2(data1, 0.1, 0.01)

# Plot
plot(c_vals, ecart, xlabel="Valeur de c", ylabel="Ecart", label="Ecart en fonction de c", legend=:bottomright)
plot!(c_vals, inf_vals, label="Borne inférieure")
plot!(c_vals, sup_vals, label="Borne supérieure")